In [3]:
import importlib
import lib.measurement_claude
import lib.validation
importlib.reload(lib.measurement_claude)
importlib.reload(lib.validation)
from lib.measurement_claude import pcqi, get_piqe, fade, average_gradient
from lib.validation import getScore_flexible


In [4]:
import os
from tqdm import tqdm  # pastikan sudah install: pip install tqdm
import csv
import cv2

def batch(
        input_folder: str, 
        output_csv: str,
        exts=(".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"),
        recursive: bool = False
        ):
    
    # 1) Ambil isi folder
    all_files = []
    walker = os.walk(input_folder) if recursive else [(input_folder, [], os.listdir(input_folder))]
    for root, _, files in walker:
        for fname in files:
            if fname.lower().endswith(exts):
                all_files.append(os.path.abspath(os.path.join(root, fname)))
    all_files.sort()
    rows = []
    tot_uiqm = 0
    tot_uciqe = 0
    tot_pcqi = 0
    tot_piqe = 0
    tot_fade = 0
    tot_ag = 0
    iter = 0
    for fpath in tqdm(all_files, desc="Processing images", unit="img"):
        filename = os.path.basename(fpath)

        filename, uiqm_score, uciqe_score = getScore_flexible(fpath, "")

        img = cv2.imread(fpath)
        # uiqm_score = uiqm(img)
        # uciqe_score = uciqe(img)
        pcqi_score = pcqi(img)
        piqe_score = get_piqe(img)
        ag_score = average_gradient(img) 
        fade_score = fade(img)

    # 2) Proses Disini

        rows.append([
            iter,
            filename,
            uiqm_score,
            uciqe_score,
            pcqi_score,
            piqe_score,
            ag_score,
            fade_score
        ])
        iter = iter + 1
        tot_uiqm = tot_uiqm + uiqm_score
        tot_uciqe = tot_uciqe + uciqe_score
        tot_pcqi = tot_pcqi + pcqi_score
        tot_piqe = tot_piqe + piqe_score
        tot_ag = tot_ag + ag_score
        tot_fade = tot_fade + fade_score
        
    length = len(rows)
    avg_uiqm = tot_uiqm / length
    avg_uciqe = tot_uciqe / length
    avg_pcqi = tot_pcqi / length
    avg_piqe = tot_piqe / length
    avg_ag = tot_ag / length
    avg_fade = tot_fade / length

    # 3) Tulis ke CSV
    write_header = True
    os.makedirs(os.path.dirname(output_csv) or ".", exist_ok=True)
    with open(output_csv, "a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        if write_header:
            writer.writerow(["id","filename", "uiqm", "uciqe", "pcqi","piqe","ag","fade"])
        writer.writerows(rows)

    print(f"Selesai. Diproses {len(rows)} "
            f"Total sekarang (CSV) ≈ {len(rows)}.")
    print(f"UIQM: {avg_uiqm}, UCIQE: {avg_uciqe}, PCQI: {avg_pcqi}, PIQE: {avg_piqe}, AG: {avg_ag}, FADE: {avg_fade}")
    return len(rows)




In [5]:
process_name = "2024_haofeng_hu_just_c" #jadi folder dan nama file csv
source_folder = "../UIEB/results/" + process_name
batch(source_folder, "hasil/"+process_name+"_metrics.csv")

Processing images: 100%|██████████| 890/890 [15:59<00:00,  1.08s/img]

Selesai. Diproses 890 Total sekarang (CSV) ≈ 890.
UIQM: 2.234600312499504, UCIQE: 2.3323336091040003, PCQI: 0.3809320757794237, PIQE: 31.6274475183551, AG: 121.47257960594604, FADE: 43.54834332761239


890

In [ ]:
# batch("../UIEB/results/2024_haofeng_hu/c", "hasil/2024_haofeng_hu_c.csv")


Processing images: 100%|██████████| 890/890 [15:39<00:00,  1.06s/img]

Selesai. Diproses 890 Total sekarang (CSV) ≈ 890.
UIQM: 2.2380007926681604, UCIQE: 2.375155354330944, PCQI: 0.3740008991877308, PIQE: 31.62516177665992, AG: 117.77420873678601, FADE: 43.715721079593436


890